In [37]:
import pandas as pd
import re
from uuid import uuid4

df=pd.read_csv('../Database.csv', low_memory=False)

df.head()

,name,add1,add2,add3,email,date,sysdate,cusno,orderno,phoff,...,sawaist,sacollar,saothers,satname,add4,mobile,mname,lname,userid,modtime
0,YASHWANTH,NaN,NaN,NaN,NaN,5/08/8202,12/08/2022,SB8633489,339/108,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DATA E O,0/01/1900
1,Y & O FASHION,NaN,NaN,NaN,NaN,21/07/7202,28/07/2022,SB8633403,339/22,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DATA E O,0/01/1900
2,JAYA,NaN,NaN,NaN,NaN,22/06/6202,5/08/2022,SB8633292,338/34,NaN,...,NaN,NaN,NaN,NaN,NaN,9566028678,NaN,MURUGAN .N,UNAIZ,0/01/1900
3,BALA,NaN,NaN,NaN,NaN,27/02/2024,15/03/2024,SB8635814,360/124,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USA,NaN,0/01/1900
4,SAJAN,NaN,NaN,NaN,NaN,26/02/2024,15/03/2024,SB8635813,360/123,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HEGDE DR,NaN,0/01/1900


In [38]:
# Clean 'orderno'
df['orderNo'] = df['orderno'].astype(str).str.strip()

# Function to clean phone numbers by removing non-digit characters for consolidation
def clean_phone(phone):
    return re.sub(r"[^\d]", "", str(phone)) if pd.notna(phone) else ""

# Create new columns for cleaned phone numbers
df['clean_mobile'] = df['mobile'].apply(clean_phone)
df['clean_phoff'] = df['phoff'].apply(clean_phone)
df['clean_phres'] = df['phres'].apply(clean_phone)

# Combine cleaned phone numbers into one field, prioritizing mobile, then office, then residential
df['combined_phone'] = df[['clean_mobile', 'clean_phoff', 'clean_phres']].bfill(axis=1).iloc[:, 0].fillna('')

# Create a unique fallback identifier based on other customer details if combined_phone is empty
df['fallback_id'] = df.apply(lambda x: f"{x['name']}-{x['mname']}-{x['lname']}-{x['email']}-{x.index}" if x['combined_phone'] == '' else x['combined_phone'], axis=1)

# Use either combined phone or fallback ID as the unique identifier
df['unique_id'] = df['fallback_id']

# Convert date to datetime format for consistency
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y', errors='coerce')

# Sort by unique_id and date in descending order
df.sort_values(by=['unique_id', 'date'], ascending=[True, False], inplace=True)

# Assign unique customer IDs based on unique_id
df['customer_id'] = pd.factorize(df['unique_id'])[0] + 1

# Group by unique_id and aggregate the first non-null value in each column
aggregated_data = df.groupby('unique_id').first().reset_index()

# Prepare customer DataFrame
customer_columns = [
    'customer_id', 'name', 'mname', 'lname', 'add1', 'add2', 'add3', 'add4', 'email', 'mobile', 'phoff', 'phres', 'date'
]
customers_final = aggregated_data[customer_columns]

customers_final.head()

,customer_id,name,mname,lname,add1,add2,add3,add4,email,mobile,phoff,phres,date
0,1,ALLS,None,CEJKA,None,None,None,None,None,0002162588578,None,None,2015-09-19
1,2,BABU(175/61),None,AMIRTHALINGAM,"11123,EAST RUN DRIVE",LAWRENCE VILLE,NEW JERSY - 08648,U.S.A,None,0012039188431,None,None,2010-05-26
2,3,VARKI,None,None,None,None,None,None,None,0013056136930,None,28362247,2007-12-25
3,4,VIVEK,A,BALASUBRAMANIYAM,None,None,None,None,None,0014152908126,None,24420234,2011-10-11
4,5,CHRIS,None,NELSON,None,None,None,None,None,001-515-707-2192,None,None,2007-10-11


In [39]:
# Ensure 'onote' field is clean
df['onote'] = df['onote'].astype(str).replace('\n', ' ', regex=True)

# Prepare the order DataFrame
order_df = df[['orderNo', 'customer_id', 'date', 'onote']].drop_duplicates(subset='orderNo', keep='first')

order_df.head()

,orderNo,customer_id,date,onote
12786,254/60,1,2015-09-19,"FM: 31.5,9,9.5,25.75,21.5 SB 2bt Side OPen"
22448,175/61,2,2010-05-26,29.5 8.5 9.25 23.25 19.5
27246,136/92,3,2007-12-25,PATTEN AVAILABLE
19817,198/107,4,2011-10-11,making 4 shirts
27819,132/117,4,2007-08-10,nan


In [40]:
#Jacket Measurement
jacketMeasurement = df[['customer_id','date','orderNo' , 'jl', 'jnl', 'jbl', 'jxback', 'jtsleeve', 'jhs', 'jchest', 'jwaist', 'scollar', 'jothers']].dropna(subset=['customer_id'])
# Drop rows with missing values in measurement columns
jacketMeasurement = jacketMeasurement.dropna(subset=['jl', 'jnl', 'jbl', 'jxback', 'jtsleeve', 'jhs', 'jchest', 'jwaist', 'scollar', 'jothers'], how='all')
jacketMeasurement['measurement_id'] = [f"{uuid4()}" for _ in range(len(jacketMeasurement))]  # Generating unique IDs

jacketMeasurement.head()

,customer_id,date,orderNo,jl,jnl,jbl,jxback,jtsleeve,jhs,jchest,jwaist,scollar,jothers,measurement_id
2993,52,2022-04-06,334/123,30,19,29,8.75,9.25,33.5,40,40,16.25,16.25,6ab86f1a-a443-44af-9a5b-c4d848e633cc
31174,166,2005-04-25,98/22,32,18,31.5,9.25,34.5,9.75,42.5,39,16,29.5,ab3f2b13-b83f-4d4e-8071-73675134f7d5
32049,185,2004-10-28,90/79,32.5,18.5,30.5,9.75,35.5,10.75,44.5,38.5,17.75,31.5,9ec92ed9-ba8c-4cd2-be7f-150575cefb9e
33318,192,2003-11-22,76/118,31.5,17.5,30.5,8.75,33.5,10,39.5,37.5,16.75,29.5,a6bf1e08-5f50-471e-bd28-644c6b8f9108
29729,357,2006-08-02,114/85,31.5,18.5,30.5,9,34,10,41.5,36.5,15.75,43,2e1c5111-5153-4954-ae10-d7d20ad555c7


In [43]:
#Shirt Measurement
shirtMeasurement = df[['customer_id','date','orderNo' , 'slength', 'sshool', 'stosleeve', 'schest', 'swaist', 'scollar', 'vcoatlen', 'sherlen', 'sothers']].dropna(subset=['customer_id'])
# Drop rows with missing values in measurement columns
shirtMeasurement = shirtMeasurement.dropna(subset=['slength', 'sshool', 'stosleeve', 'schest', 'swaist', 'scollar', 'vcoatlen', 'sherlen', 'sothers'], how='all')
shirtMeasurement['measurement_id'] = [f"{uuid4()}" for _ in range(len(shirtMeasurement))]  # Generating unique IDs

shirtMeasurement.head()

,customer_id,date,orderNo,slength,sshool,stosleeve,schest,swaist,scollar,vcoatlen,sherlen,sothers,measurement_id
27819,4,2007-08-10,132/117,29,18.5,24.5,NaN,NaN,15,NaN,NaN,NaN,f7983663-70ac-4977-a201-1bcc5207f76b
27574,5,2007-10-11,134/71,31.5,20,27.5,NaN,NaN,16.75,NaN,NaN,NaN,679afb45-1b07-48ed-a959-6cb028010eb4
28477,6,2007-04-11,125/72,32.5,21,27,NaN,NaN,17,NaN,NaN,NaN,67dee849-5632-4fb5-ad53-32ff55e65412
28548,7,2007-03-26,124/117,29.5,9.5,32.5,NaN,NaN,15.5,NaN,NaN,NaN,4b2c03a7-7375-464b-a755-a682cd651275
19874,8,2011-09-29,196/124,30,10,34,NaN,NaN,16.25,NaN,NaN,NaN,a0b48fbd-94d8-4627-80c3-c8af231ac894


In [42]:
#Pant Measurement
pantMeasurement = df[['customer_id','date','orderNo' , 'plength', 'pinseem', 'pwaist', 'phips', 'pbottom', 'pknee', 'pothers']].dropna(subset=['customer_id'])
# Drop rows with missing values in measurement columns
pantMeasurement = pantMeasurement.dropna(subset=['plength', 'pinseem', 'pwaist', 'phips', 'pbottom', 'pknee', 'pothers'], how='all')
pantMeasurement['measurement_id'] = [f"{uuid4()}" for _ in range(len(pantMeasurement))]  # Generating unique IDs

pantMeasurement.head()

,customer_id,date,orderNo,plength,pinseem,pwaist,phips,pbottom,pknee,pothers,measurement_id
1101,665,2023-05-25,350/75,39,29.5,33.5,39.5,14,9,12.75,9860866c-d65e-46fd-8b2f-1125143a2e33
2606,1132,2022-06-18,338/20,42.5,32.5,36.5,40.5,16,9.25,13.25,c14bca49-89b9-4c79-875e-b7d019cdd082
2229,1351,2022-09-15,341/35,41.5,30,38.5,43,15,18,26,b3d948ae-4331-4051-b2ce-a993b2c592ee
3313,1400,2022-01-22,332/48,31,18,37,33,8.75,30,9.25,a1b09480-1eab-46a5-a366-482597a010cc
2537,1839,2022-07-09,338/90,39.5,29.5,38,42,16,9.75,13.75,f008a7e0-1bf8-48f2-b12a-a298e1c1993c
